# Step 1: Install and Import Required Libraries
# python-docx: used to read content from Word (.docx) files.
# transformers: provides pretrained NLP models from Hugging Face.
# sentence-transformers: makes it easy to generate embeddings (vector representations of text).
# faiss-cpu: a fast library for similarity search (finding the most relevant text chunks).
# gradio: used to create an easy-to-use web interface for the chatbot.
# After installing, we import the libraries into Python so we can use them later.



In [1]:
!pip install python-docx transformers sentence-transformers faiss-cpu gradio --quiet

from docx import Document
import re
from sentence_transformers import SentenceTransformer
import faiss
from transformers import pipeline
import gradio as gr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 35.9 MB/s eta 0:00:00


# Step 2: Upload the Word Document
# In Colab, we can upload local files from our computer.
# files.upload() opens a file picker to choose the file.
# The uploaded file is stored in the "uploaded" dictionary.
# We extract the file name (key of the dictionary) and store it in "filename".
# This filename will be used later when we open and read the Word file.

In [2]:
# Upload Word file
from google.colab import files
uploaded = files.upload()
filename = list(uploaded.keys())[0]

Saving Tim Cook.docx to Tim Cook.docx


#  Step 3: Extract Text from the Word File
# We use Document(filename) from python-docx to open the Word file.
# Each paragraph in the Word file is stored in doc.paragraphs.
# I remove empty lines using .strip() and only keep paragraphs with text.
# Finally, we join all cleaned paragraphs into one single string called "text".
# This will give us the full book content in plain text format.

In [3]:
# Extract text from Word
doc = Document(filename)
full_text = []
for para in doc.paragraphs:
    if para.text.strip():
        full_text.append(para.text.strip())
text = "\n".join(full_text)


# Step 4: Split the Text into Smaller Chunks
# Why? Because long text is hard to process for embeddings and retrieval.
# The split_text function divides the text into smaller pieces ("chunks").
# First, we split text into sentences using regular expressions.
#  We keep adding sentences to a "chunk" until it reaches a max length (default: 300 characters).
# Once the chunk is full, we save it and start a new one.
# At the end, all chunks are returned as a list.
# This ensures the text is broken into manageable, meaningful pieces for search.

In [4]:
# Split into chunks for better retrieval
def split_text(text, max_len=300):
    sentences = re.split(r'(?<=[.!?]) +', text)
    chunks = []
    chunk = ""
    for sentence in sentences:
        if len(chunk) + len(sentence) < max_len:
            chunk += " " + sentence
        else:
            chunks.append(chunk.strip())
            chunk = sentence
    if chunk:
        chunks.append(chunk.strip())
    return chunks

chunks = split_text(text)

# Step 5: Create Embeddings and Build FAISS Index
# SentenceTransformer('all-MiniLM-L6-v2'): this is a small but powerful model
# that converts text chunks into numerical vectors (embeddings).
# We generate embeddings for each chunk and store them in "embeddings".
# Each embedding is a high-dimensional vector that captures the meaning of the text.
# dimension = number of features in each vector (e.g., 384).
# FAISS IndexFlatL2: a simple similarity search index based on L2 distance (Euclidean).
# index.add(embeddings): we add all the embeddings to the FAISS index.
# Later, when a user asks a question, we can search this index to find the most relevant chunks.

In [5]:
# Create embeddings and build FAISS index
embedder = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = embedder.encode(chunks, convert_to_numpy=True)

dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

#  Step 6: Load the Hugging Face Question-Answering Pipeline
# -We use Hugging Face's "pipeline" function to load a ready-to-use model for Q&A tasks.
# The task name is "question-answering", which means the model can take:
#     (1) a question (from the user)
#     (2) a context (a piece of text from our book)
#   and then find the best possible answer from the context.
# Model used here: "distilbert-base-uncased-distilled-squad"
#     A lightweight version of BERT, trained on the SQuAD dataset (Stanford Q&A dataset).
#     Fast and efficient for extracting answers from text passages.
# This pipeline will later be combined with FAISS search:
#     FAISS finds the most relevant chunk of text,
#     and the QA pipeline extracts the exact answer from that chunk.

In [6]:
# Load Hugging Face QA pipeline
qa_pipeline = pipeline("question-answering", model="distilbert-base-uncased-distilled-squad")

config.json:   0%|          | 0.00/451 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/265M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Device set to use cpu


Converts the user’s question into a numerical embedding.

Searches the FAISS index to find the top 3 relevant text chunks.

Combines these chunks into a single context string.

Uses a QA model to extract the answer from the context.

Returns the answer as a string.

In [7]:
# Query function to retrieve top chunks and answer
def answer_question(question):
    q_emb = embedder.encode([question], convert_to_numpy=True)
    D, I = index.search(q_emb, k=3)  # Top 3 chunks
    context = " ".join([chunks[i] for i in I[0]])
    result = qa_pipeline(question=question, context=context)
    return result['answer']

In [9]:
# Gradio interface
iface = gr.Interface(fn=answer_question, inputs="text", outputs="text",
                     title="Chatbot Made by Shaifur Rahman",
                     description="Ask questions based on the uploaded textbook.")

iface.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://fb5083c42d5a0c276f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
